## Lines from `Grid` objects

Grids that have been post-processed through a photoionisation code (e.g. `Cloudy`) contain information on emission lines. We can see what lines are available on a grid by printing the ``available_lines`` attribute.

In [ ]:
import numpy as np

from synthesizer import Grid

grid = Grid("test_grid.hdf5")

print(grid.available_lines)

This is also reported if we give use the `print` function on a grid directly:

In [ ]:
print(grid)

### Extracting lines from a Grid

To demonstrate, we choose some age and metallicity and extract the spectra at that grid point. We can then get information on a single line, in this case H-$\beta$.

In [ ]:
log10age = 6.0  # log10(age/yr)
metallicity = 0.01

# find nearest grid point
grid_point = grid.get_grid_point(log10ages=log10age, metallicity=metallicity)
print(grid_point)
line = grid.get_lines(grid_point, "H 1 6562.80A")
print(line)

We can do this for a combination of lines (e.g. a doublet) if we just pass a comma-separated list of lines ids to the ``line_id`` argument.

In [ ]:
from synthesizer.emissions.utils import (
    Hb,
    O3b,
    O3r,
)

line = grid.get_lines(grid_point, ", ".join([Hb, O3r, O3b]))
print(line)

We can also get a collection of individual lines by passing a list of line ids to the ``line_id`` argument. 

In [ ]:
line = grid.get_lines(grid_point, [Hb, O3r, O3b])
print(line)

If we don't pass a list then it defaults to returning all available lines.


In [ ]:
lines = grid.get_lines(grid_point)
print(lines)

### Ratios as a function of metallicity

To show the dependence on stellar metallicity we can loop over the metallicity grid:

In [ ]:
import matplotlib.pyplot as plt

ratio_id = "R23"
ia = 0  # 1 Myr old for test grid
ratios = []
for iZ, Z in enumerate(grid.metallicity):
    grid_point = (ia, iZ)
    lines = grid.get_lines(grid_point)
    ratios.append(lines.get_ratio(ratio_id))

Zsun = grid.metallicity / 0.0124
plt.plot(Zsun, ratios)
plt.xlim([0.01, 1])
plt.ylim([1, 20])
plt.xscale("log")
plt.yscale("log")
plt.xlabel(r"$Z/Z_{\odot}$")
plt.ylabel(rf"{ratio_id}")
plt.show()

### Line Diagrams as a function of metallicity

We can also generate diagrams using pairs of line ratios, such as the famous Baldwin, Phillips & Terlevich (BPT) diagram.

``line_ratios`` also contains some classification regions (e.g. [Kewley+13](https://ui.adsabs.harvard.edu/abs/2013ApJ...774L..10K/abstract) and [Kauffmann+03](https://ui.adsabs.harvard.edu/abs/2003MNRAS.346.1055K/abstract)) that we can plot:

In [ ]:
from synthesizer.emissions import line_ratios

diagram_id = "BPT-NII"
ia = 0  # 1 Myr old for test grid
x = []
y = []
for iZ, Z in enumerate(grid.metallicity):
    grid_point = (ia, iZ)
    lines = grid.get_lines(grid_point)
    x_, y_ = lines.get_diagram(diagram_id)
    x.append(x_)
    y.append(y_)


# Plot the Kewley SF/AGN dividing line
fig, ax = plt.subplots()
ax.plot(x, y)
logNII_Ha = np.arange(-2.0, 1.0, 0.01)
logOIII_Hb = line_ratios.plot_bpt_kewley01(
    logNII_Ha, fig=fig, ax=ax, show=True, c="k", lw="2", alpha=0.3
)